<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-04 11:24:04
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.37 C
-------------------
Today PnL: 29.73 K (0.22%)
Current PnL: -13.07 L (-9.1%)
CY Booked + Current PnL: -6.33 L (-4.41%)
-------------------
Total profit:  4.73 L
Total loss:  -17.80 L
-------------------
Total Booked + Current PnL: 20.65 L (17.72%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.93%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 71.48 L (52.24%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.9%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.00,54.03,M-SC,94.79,133461.0,-9018.0,9636.0,0.63,-6.33,7.22,0.44,243.0,-0.94,0.98,57.52,OX40N,NTT,BANKS
3,ADANIPORTS,1103.69,1583.00,1.31,M-LC,33.33,101672.0,23310.0,10716.0,0.06,29.75,10.54,43.43,77.0,2.18,0.74,36.65,XY24,NTT,MISC
39,HINDALCO,651.95,761.55,-19.08,H-LC,85.42,111120.0,6808.0,10723.0,0.24,6.53,9.65,16.81,27.0,0.63,0.81,23.62,X5K,ATH,METALS
5,APOLLOHOSP,6904.43,8285.00,-16.74,H-LC,86.46,113370.0,9804.0,10906.0,-0.09,9.47,9.62,20.00,30.0,0.90,0.83,24.87,X40N,BTT,HEALTHCARE
88,UNIONBANK,123.87,163.00,0.15,M-LC,29.17,173336.0,32496.0,11995.0,0.57,23.07,6.92,31.59,89.0,2.71,1.27,50.57,XY24,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.0,-31.29,M-MC,19.79,181576.0,1768.0,110652.0,-0.36,0.98,60.94,62.52,171.0,0.02,1.33,35.41,XR,NTT,BANKS
55,LALPATHLAB,2804.19,3545.0,-32.31,H-SC,27.08,155700.0,1470.0,39283.0,2.69,0.95,25.23,26.42,125.0,0.04,1.14,22.78,X40N,NTT,HEALTHCARE


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,10.42,49242.0,7987.0,2817.0,5.30,19.36,5.72,26.19,1.0,2.84,0.36,15.93,XY24,ATH,IT
65,RELAXO,902.64,1176.00,-35.51,H-SC,61.46,58349.0,-57189.0,92180.0,4.17,-49.50,157.98,30.28,134.0,-0.62,0.43,13.99,X40N,NTT,FOOTWEAR
70,SATIN,199.77,274.00,-14.59,H-SC,87.50,168026.0,-27948.0,100765.0,2.89,-14.26,59.97,37.16,130.0,-0.28,1.23,25.48,XY24,NTT,FINANCE
55,LALPATHLAB,2804.19,3545.00,-32.31,H-SC,27.08,155700.0,1470.0,39283.0,2.69,0.95,25.23,26.42,125.0,0.04,1.14,22.78,X40N,NTT,HEALTHCARE
78,SURYODAY,189.78,240.00,57.89,H-SC,12.50,111998.0,-37928.0,77603.0,2.34,-25.30,69.29,26.46,145.0,-0.49,0.82,43.48,XR,NTT,BANKS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SAMMAANCAP,170.35,326.00,-164.77,M-SC,40.10,74754.0,-27456.0,120847.0,-8.21,-26.86,161.66,91.37,206.0,-0.23,0.55,21.35,XY25,NTT,FINANCE
11,BANDHANBNK,214.25,400.00,-29.71,H-SC,70.83,208812.0,-39718.0,255189.0,-2.13,-15.98,122.21,86.70,129.0,-0.16,1.53,38.14,XY24,NTT,BANKS
74,SIEMENS,4406.90,7969.85,-3.50,X-LC,23.96,81492.0,-28680.0,117756.0,-1.57,-26.03,144.50,80.85,14.0,-0.24,0.60,20.03,AR,ATH,ELECTRICAL
4,ALKYLAMINE,2347.98,4546.37,6.59,X-SC,97.92,99218.0,-1745.0,96271.0,-1.44,-1.73,97.03,93.63,64.0,-0.02,0.73,47.78,SR,ATH,CHEMICALS
45,INDIAMART,2327.09,4850.92,-48.26,H-SC,78.12,134827.0,11491.0,122275.0,-1.40,9.32,90.69,108.45,122.0,0.09,0.99,33.75,AR,ATH,MISC


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,54.03,M-SC,94.79,133461.0,-9018.0,9636.0,0.63,-6.33,7.22,0.44,243.0,-0.94,0.98,57.52,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-30.32,M-SC,11.46,148554.0,-22587.0,22536.0,0.43,-13.20,15.17,-0.03,189.0,-1.00,1.09,6.77,OX40N,NTT,DURABLES
92,VIPIND,488.80,489.0,-212.91,H-SC,95.83,81461.0,-13855.0,13897.0,1.14,-14.54,17.06,0.04,154.0,-1.00,0.60,60.67,OX40N,NTT,MISC
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,89.58,148800.0,-25759.0,25787.0,-0.11,-14.76,17.33,0.02,127.0,-1.00,1.09,29.03,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-16.97,X-MC,17.71,87136.0,-15516.0,15510.0,0.14,-15.12,17.80,-0.00,58.0,-1.00,0.64,32.46,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,10.42,49242.0,7987.0,2817.0,5.30,19.36,5.72,26.19,1.0,2.84,0.36,15.93,XY24,ATH,IT
83,TCS,3794.03,4998.00,-18.06,X-LC,41.67,266651.0,-29283.0,123193.0,0.52,-9.90,46.20,31.73,1.0,-0.24,1.95,6.73,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.55,X-LC,59.38,290338.0,28737.0,116890.0,0.20,10.98,40.26,55.67,2.0,0.25,2.12,17.76,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-16.87,X-LC,26.04,260322.0,-10025.0,44645.0,0.47,-3.71,17.15,12.81,6.0,-0.22,1.90,8.87,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-12.34,X-LC,28.12,278004.0,12578.0,44342.0,-0.10,4.74,15.95,21.44,9.0,0.28,2.03,11.70,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,0.15,M-LC,29.17,173336.0,32496.0,11995.0,0.57,23.07,6.92,31.59,89.0,2.71,1.27,50.57,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,1.31,M-LC,33.33,101672.0,23310.0,10716.0,0.06,29.75,10.54,43.43,77.0,2.18,0.74,36.65,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,77.08,73872.0,39423.0,524.0,1.11,114.44,0.71,115.97,NaN,75.23,0.54,12.96,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,95.22,M-MC,55.21,234330.0,9368.0,158290.0,-0.31,4.16,67.55,74.53,185.0,0.06,1.71,36.05,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.79,25.50,H-SC,40.10,128596.0,3299.0,55811.0,-0.57,2.63,43.40,47.18,139.0,0.06,0.94,60.88,MH,ATH,POWER
20,CAMS,3643.00,5250.99,6.09,H-SC,48.44,119179.0,17175.0,27852.0,-0.05,16.84,23.37,44.14,124.0,0.62,0.87,35.81,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,1.31,M-LC,33.33,101672.0,23310.0,10716.0,0.06,29.75,10.54,43.43,77.0,2.18,0.74,36.65,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,29.17,173336.0,32496.0,11995.0,0.57,23.07,6.92,31.59,89.0,2.71,1.27,50.57,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,77.08,73872.0,39423.0,524.0,1.11,114.44,0.71,115.97,NaN,75.23,0.54,12.96,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-37.67,M-SC,90.62,205668.0,34047.0,128028.0,-0.19,19.84,62.25,94.43,198.0,0.27,1.50,51.80,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.86,M-LC,83.33,166439.0,15494.0,93955.0,0.52,10.26,56.45,72.51,69.0,0.16,1.22,14.65,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-48.26,H-SC,78.12,134827.0,11491.0,122275.0,-1.40,9.32,90.69,108.45,122.0,0.09,0.99,33.75,AR,ATH,MISC
54,KPIGREEN,497.21,731.79,25.50,H-SC,40.10,128596.0,3299.0,55811.0,-0.57,2.63,43.40,47.18,139.0,0.06,0.94,60.88,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,4.17,17463.0,-32130.0,1053.0,1.22,-64.79,6.03,-62.66,NaN,-30.51,0.13,17.11,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,67834.0,-45715.0,85769.0,0.71,-40.26,126.44,35.27,261.0,-0.53,0.50,66.02,XR,NTT,HOTELS
7,ASIANTILES,75.41,137.00,7200.00,M-SC,88.54,80076.0,-13734.0,90350.0,0.85,-14.64,112.83,81.67,242.0,-0.15,0.59,55.07,XR,NTT,CERAMICS
58,MASFIN,326.60,399.50,-18.70,H-SC,73.96,92595.0,-5385.0,27251.0,0.41,-5.50,29.43,22.32,137.0,-0.20,0.68,34.29,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,6.59,X-SC,97.92,99218.0,-1745.0,96271.0,-1.44,-1.73,97.03,93.63,64.0,-0.02,0.73,47.78,SR,ATH,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-28.28,L-SC,100.00,143963.0,2121.0,54447.0,1.82,1.50,37.82,39.88,259.0,0.04,1.05,45.89,XY24,NTT,POWER
15,BLUESTARCO,1646.70,2337.55,-0.83,H-SC,98.96,185980.0,21310.0,47778.0,0.95,12.94,25.69,41.95,121.0,0.45,1.36,21.37,X40N,ATH,AC
4,ALKYLAMINE,2347.98,4546.37,6.59,X-SC,97.92,99218.0,-1745.0,96271.0,-1.44,-1.73,97.03,93.63,64.0,-0.02,0.73,47.78,SR,ATH,CHEMICALS
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,67834.0,-45715.0,85769.0,0.71,-40.26,126.44,35.27,261.0,-0.53,0.50,66.02,XR,NTT,HOTELS
92,VIPIND,488.80,489.00,-212.91,H-SC,95.83,81461.0,-13855.0,13897.0,1.14,-14.54,17.06,0.04,154.0,-1.00,0.60,60.67,OX40N,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.85
1,25,41.81
2,50,70.44


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.52
LC    33.86
MC    21.60
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.51
X40      12.24
X40N     11.95
XY25     11.92
XR       10.15
OX40N     8.26
AR        8.21
MH        1.69
X5K       1.46
X200      1.40
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.70
M-SC    15.97
H-LC    14.11
X-LC    12.09
H-MC     9.54
X-MC     7.60
M-LC     6.62
X-SC     4.39
M-MC     4.10
L-SC     2.46
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.02,-1.73,35.77
IT,12.10,-9.87,63.82
FINANCE,7.88,-12.05,52.53
BANKS,7.85,-6.38,49.79
MISC,6.62,-14.90,68.10
PAINTS,5.72,-11.63,33.63
ELECTRICAL,5.51,2.26,43.40
HEALTHCARE,4.57,-3.16,33.29
AUTO,4.34,-13.86,63.92


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2691338.0,27
XR,1072131.0,13
AR,1021772.0,10
X40,645425.0,9
X40N,528438.0,11
XY25,486697.0,8
OX40N,302522.0,11
SR,177574.0,2
MH,85024.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1956019.0,20
M-SC,1757853.0,21
H-LC,610110.0,12
H-MC,530536.0,8
X-LC,520384.0,7
X-MC,411763.0,6
M-MC,409381.0,3
X-SC,318364.0,5
M-LC,267682.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      870243.0      6
M-SC       XY24      811204.0      7
H-SC       AR        522056.0      3
M-SC       XR        331359.0      4
M-MC       XY24      298729.0      2
H-MC       XY24      298272.0      3
X-LC       X40       240083.0      2
H-SC       XR        212257.0      3
           X40N      207093.0      4
H-LC       X40N      194270.0      4
M-SC       AR        182679.0      2
           OX40N     159195.0      5
M-LC       XY24      158826.0      3
H-LC       X40       155316.0      3
X-MC       X40       147578.0      2
L-SC       XR        147170.0      2
X-SC       XR        123041.0      1
M-SC       XY25      120847.0      1
X-LC       XY25      118419.0      3
           AR        117756.0      1
M-MC       XR        110652.0      1
X-MC       XY25      103479.0      1
X-SC       SR         96271.0      1
M-LC       XR         93955.0      1
X-MC       XY24       93179.0      1
H-SC       OX40N      88559.0      3
H-MC       XY25       82203.0      1
M-SC       SR         81303.0      1
H-LC       AR         80698.0      1
M-SC       X40        71266.0      1
H-LC       X200       69445.0      1
           X5K        67607.0      2
X-SC       XY24       59794.0      1
L-SC       AR         59325.0      1
H-MC       AR         58734.0      1
H-SC       MH         55811.0      1
L-SC       XY24       54447.0      1
L-MC       XR         53697.0      1
X-MC       X40N       52017.0      1
L-LC       XY25       46848.0      1
X-LC       X40N       44126.0      1
H-LC       XY24       42774.0      1
X-SC       OX40N      39258.0      2
H-MC       X40        31182.0      1
           X40N       30932.0      1
           MH         29213.0      1
X-MC       OX40N      15510.0      1
M-LC       XY25       14901.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
